In [1]:
from cocotb.binary import BinaryValue

In [2]:
import json
import ast
from cocotb.binary import BinaryValue

def create_testbench_json(stimulus_file, output_file, output_json_file):
    """
    将stimulus.json和our_output.txt合并为一个完整的testbench.json文件
    
    参数:
    stimulus_file -- stimulus.json文件路径
    output_file -- our_output.txt文件路径
    output_json_file -- 输出的testbench.json文件路径
    """
    # 读取stimulus.json获取输入数据
    with open(stimulus_file, 'r') as f:
        stimulus_data = json.load(f)
    
    # 读取our_output.txt获取输出数据
    with open(output_file, 'r') as f:
        output_lines = f.readlines()
    
    # 解析输出数据
    all_outputs = []
    for line in output_lines:
        try:
            # 将字符串转换为Python对象
            parsed = ast.literal_eval(line)
            if parsed[0] and 'out' in parsed[1]:
                # 解析JSON字符串
                output_data = json.loads(parsed[1]['out'])
                # 将二进制字符串转换为整数（如果需要）
                for scenario in output_data:
                    for output_var in scenario['output variable']:
                        for key in output_var:
                            # 保持字符串格式，不转换为整数
                            pass
                all_outputs.append(output_data)
        except Exception as e:
            print(f"解析输出数据时出错：{e}")
            continue
    
    # 如果没有有效输出，退出
    if not all_outputs:
        print("没有找到有效的输出数据")
        return
    
    # 检查所有输出组中是否存在不一致
    # 只使用第一组输出作为标准结果
    standard_output = all_outputs[0]
    
    # 合并输入和输出
    combined_data = []
    
    # 遍历每个测试场景
    for stimulus_scenario in stimulus_data:
        scenario_name = stimulus_scenario['scenario']
        
        # 在输出数据中查找对应的场景
        output_scenario = next((s for s in standard_output if s['scenario'] == scenario_name), None)
        
        if output_scenario:
            # 创建合并后的场景数据
            combined_scenario = {
                "scenario": scenario_name,
                "input variable": stimulus_scenario['input variable'],
                "output variable": output_scenario['output variable']
            }
            combined_data.append(combined_scenario)
        else:
            print(f"警告：在输出数据中未找到场景 '{scenario_name}'")
            # 添加没有输出的场景
            combined_scenario = {
                "scenario": scenario_name,
                "input variable": stimulus_scenario['input variable'],
                "output variable": []
            }
            combined_data.append(combined_scenario)
    
    # 将合并后的数据写入JSON文件
    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(combined_data, f, indent=2, ensure_ascii=False)
    
    print(f"已成功将stimulus和output数据合并到 {output_json_file}")


In [ ]:
taskids=[58]
for taskid in taskids:
    create_testbench_json(f'{taskid}/stimulus.json', f'{taskid}/our_output.txt', f'{taskid}/testbench.json')

已成功将stimulus和output数据合并到 53/testbench.json
已成功将stimulus和output数据合并到 54/testbench.json


In [12]:
import json
import ast
from cocotb.binary import BinaryValue

def compare_scenarios(taskid):
    flag=True
    # 读取文件内容
    with open(f'{taskid}/our_output.txt', 'r') as f:
        
        lines = f.readlines()
    
    # 解析每行的数据
    all_test_cases = []
    for line in lines:
        try:
            # 将字符串转换为Python对象
            parsed = ast.literal_eval(line)
            if parsed[0] and 'out' in parsed[1]:
                # 解析JSON字符串
                test_cases = json.loads(parsed[1]['out'])
                for ke in test_cases:
                    for m in ke['output variable']:
                        for key in m:
                            # 将二进制字符串转换为整数
                            m[key] = int(BinaryValue(m[key]).integer)
                all_test_cases.append(test_cases)
        except:
            continue
    
    # 按scenario分组比较
    inconsistencies = {}
    
    # 遍历每个scenario类型
    for test_case in all_test_cases[0]:  # 使用第一组数据作为基准
        scenario = test_case['scenario']
        inconsistencies[scenario] = []
    
    # 两两比较所有组合
    for i in range(len(all_test_cases)):
        for j in range(i+1, len(all_test_cases)):
            # 比较第i组和第j组
            for test_case in all_test_cases[i]:
                scenario = test_case['scenario']
                # 在第j组中找到相同scenario的数据
                scenario_data_j = next((x for x in all_test_cases[j] if x['scenario'] == scenario), None)
                
                if scenario_data_j:
                    # 比较output variable
                    if str(test_case['output variable']) != str(scenario_data_j['output variable']):
                        inconsistencies[scenario].append({
                            'group_pair': [i, j],
                            'data_i': test_case['output variable'],
                            'data_j': scenario_data_j['output variable']
                        })
    
    # 检查不一致性
    print(taskid,"Diff：")
    for scenario, data_list in inconsistencies.items():
        if data_list:  # 只输出有不一致的scenario
            flag=False
            print(f"\nScenario: {scenario}")
            for data in data_list:
                print(f"Group {data['group_pair'][0]} vs Group {data['group_pair'][1]}:")
                print(f"组 {data['group_pair'][0]}: {str(data['data_i'])}")
                print(f"组 {data['group_pair'][1]}: {str(data['data_j'])}")
    if flag:
                create_testbench_json(f'{taskid}/stimulus.json', f'{taskid}/our_output.txt', f'{taskid}/testbench.json')
                print(taskid,"No diff")
    return flag
result={}
cases=[45]
for i in cases  :
    result[i]=compare_scenarios(i)
print(result)
not_success=[]
success=[]
for i in cases:
    if result[i]==False:
        not_success.append(i)
    else:
        success.append(i)
print(not_success)
print(success)

45 Diff：

Scenario: Valid Inputs for Logic 1
Group 0 vs Group 1:
组 0: [{'out_sop': 0, 'out_pos': 0}, {'out_sop': 0, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
组 1: [{'out_sop': 1, 'out_pos': 1}, {'out_sop': 1, 'out_pos': 1}, {'out_sop': 1, 'out_pos': 1}]
Group 0 vs Group 2:
组 0: [{'out_sop': 0, 'out_pos': 0}, {'out_sop': 0, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
组 2: [{'out_sop': 1, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}, {'out_sop': 1, 'out_pos': 1}]
Group 0 vs Group 3:
组 0: [{'out_sop': 0, 'out_pos': 0}, {'out_sop': 0, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
组 3: [{'out_sop': 1, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
Group 0 vs Group 4:
组 0: [{'out_sop': 0, 'out_pos': 0}, {'out_sop': 0, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
组 4: [{'out_sop': 0, 'out_pos': 1}, {'out_sop': 1, 'out_pos': 0}, {'out_sop': 1, 'out_pos': 1}]
Group 1 vs Group 2:
组 1: [{'out_sop': 1, 'out_pos': 1}, {'out_sop': 1, 'out_pos': 1}, {'out_sop': 1, 'out_p

In [21]:
with open('summary.txt', 'r') as f:
    summary=f.read()
ambiguous=[]
for i,line in enumerate(summary.split('\n')):
    if 'unambiguous' in line:
        ambiguous.append(i)
print(ambiguous)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 66, 68, 73, 74, 75, 76, 78, 80, 81, 82, 83, 86, 87, 88, 89, 90, 91, 96, 97, 99, 100, 101, 103, 104, 105, 106, 107, 108, 111, 112, 113, 114, 115, 117, 119, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 159, 161, 162, 163, 165, 168, 170, 171, 172, 179, 186, 188, 191, 193, 194, 195, 198, 199, 202, 209, 210, 212, 215, 216, 219]


In [ ]:
[31, 33, 36, 38, 39, 41, 42, 43, 44, 48, 49, 50, 51, 54, 55, 56, 57, 61, 63, 64, 65, 67, 68, 70, 71, 72, 73, 74, 75, 76, 78, 83, 87, 89, 90, 91, 94, 95, 96, 100, 101, 102, 103, 104, 105, 107, 108, 110, 112, 113, 115, 116, 120, 122, 123, 124, 125, 128, 129, 130, 131, 132, 133, 136, 137, 138, 139, 151]

In [23]:
len(not_success)

58

In [4]:
print(not_success)

[32, 34, 35, 37, 40, 45, 46, 47, 52, 53, 58, 59, 60, 62, 66, 69, 77, 79, 80, 81, 82, 84, 85, 86, 88, 92, 93, 97, 98, 99, 106, 109, 111, 114, 117, 118, 119, 121, 126, 127, 134, 135, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156]


In [10]:
mage=[31, 34, 45, 46, 47, 48, 54, 62, 63, 66, 68, 70, 74, 78, 79, 80, 82, 86, 88, 89, 93, 96, 98, 99, 104, 109, 110, 112, 113, 116, 117, 122, 124, 127, 128, 129, 131, 133, 134, 135, 136, 137, 139, 140, 141, 142, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156]
not_in_mage_in_not_success=[]
for i in not_success:
    if i not in mage:
        not_in_mage_in_not_success.append(i)
in_mage_not_in_not_success=[]
for i in mage:  
    if i not in not_success:
        in_mage_not_in_not_success.append(i)
in_mage_in_not_success=[]
for i in mage:
    if i in not_success:
        in_mage_in_not_success.append(i)
print(len(not_in_mage_in_not_success))
print(len(mage))
print(len(in_mage_not_in_not_success))
print(len(in_mage_in_not_success))

26
57
25
32


In [9]:
print(not_in_mage_in_not_success)

[32, 35, 37, 40, 52, 53, 58, 59, 60, 69, 77, 81, 84, 85, 92, 97, 106, 111, 114, 118, 119, 121, 126, 143, 144, 154]


In [11]:
print(in_mage_in_not_success)

[34, 45, 46, 47, 62, 66, 79, 80, 82, 86, 88, 93, 98, 99, 109, 117, 127, 134, 135, 140, 141, 142, 145, 146, 147, 148, 149, 150, 152, 153, 155, 156]


In [7]:
a=[1,2,3]
a[1:2]

[2]

In [8]:
import json
import ast

def compare_scenarios():
    # 读取文件内容
    with open('31/our_output.txt', 'r') as f:
        lines = f.readlines()
    
    # 解析每行的数据
    all_test_cases = []
    for line in lines:
        try:
            # 将字符串转换为Python对象
            parsed = ast.literal_eval(line)
            if parsed[0] and 'out' in parsed[1]:
                # 解析JSON字符串
                test_cases = json.loads(parsed[1]['out'])
                all_test_cases.append(test_cases)
        except:
            continue
    
    # 按scenario分组比较
    inconsistencies = {}
    for test_case in all_test_cases[0]:
        scenario = test_case['scenario']
        inconsistencies[scenario] = []
    
    for group_base, base_group in enumerate(all_test_cases[:-1]):
        for test_case in base_group:  
            
            
            # 获取所有测试组中该scenario的数据
            for group_idx, test_group in enumerate(all_test_cases[group_base+1:]):
                scenario_data = next((x for x in test_group if x['scenario'] == scenario), None)
                if scenario_data:
                    inconsistencies[scenario].append({
                        'group': [group_base,group_idx],
                        'data': scenario_data['output variable']
                    })
        
    # 检查不一致性
    print("发现的不一致：")
    for scenario, data_list in inconsistencies.items():
        
        for data in data_list[1:]:
            if str(data['data']) != reference:
                print(f"\nScenario: {scenario}")
                print(f"组 0: {reference}")
                print(f"组 {data['group']}: {str(data['data'])}")

# 执行比较
compare_scenarios()

发现的不一致：


NameError: name 'reference' is not defined

In [6]:
import json
import ast

def compare_scenarios():
    # 读取文件内容
    with open('32/our_output.txt', 'r') as f:
        lines = f.readlines()
    
    # 解析每行的数据
    all_test_cases = []
    for line in lines:
        try:
            # 将字符串转换为Python对象
            parsed = ast.literal_eval(line)
            if parsed[0] and 'out' in parsed[1]:
                # 解析JSON字符串
                test_cases = json.loads(parsed[1]['out'])
                for ke in test_cases:
                    for m in ke:
                        m.value=int(BinaryValue(m).integer)
                all_test_cases.append(test_cases)
        except:
            continue
    
    # 按scenario分组比较
    inconsistencies = {}
    
    # 遍历每个scenario类型
    for test_case in all_test_cases[0]:  # 使用第一组数据作为基准
        scenario = test_case['scenario']
        inconsistencies[scenario] = []
        
        # 获取所有测试组中该scenario的数据
        for group_idx, test_group in enumerate(all_test_cases):
            scenario_data = next((x for x in test_group if x['scenario'] == scenario), None)
            if scenario_data:
                inconsistencies[scenario].append({
                    'group': group_idx,
                    'data': scenario_data['output variable']
                })
    
    # 检查不一致性
    print("发现的不一致：")
    for scenario, data_list in inconsistencies.items():
        reference = str(data_list[0]['data'])  # 第一组作为参考
        for data in data_list[1:]:
            if str(data['data']) != reference:
                print(f"\nScenario: {scenario}")
                print(f"组 0: {reference}")
                print(f"组 {data['group']}: {str(data['data'])}")

# 执行比较
compare_scenarios()

IndexError: list index out of range